In [1]:
# install necessary packages
%pip install azure-storage-blob python-dotenv

In [2]:
# import necessary libraries
import pandas as pd
import os
from azure.storage.blob import BlobServiceClient
from dotenv import load_dotenv

In [3]:
# Data Extraction
try:
    data = pd.read_csv(r'data\zipco_transaction.csv')
    print('Data extracted successfully')
except Exception as e:
    print(f'An error occured: {e}')


Data extracted successfully


In [4]:
data.head()

,Date,ProductName,Quantity,UnitPrice,StoreLocation,PaymentType,PromotionApplied,Weather,Temperature,StaffPerformanceRating,...,DeliveryTime_min,OrderType,CustomerName,CustomerAddress,Customer_PhoneNumber,CustomerEmail,Staff_Name,Staff_Email,DayOfWeek,TotalSales
0,2023-01-01 00:00:00,Vanilla Cake,2,12.532304,South,Cash,True,Rainy,20.654914,Poor,...,30,In Store,William Adams,"9851 David Green\nTonyaburgh, VA 02853",(916)427-7276x861,lisa00@example.net,John Bridges,pdavidson@example.com,Sunday,25.064608
1,2023-01-01 01:00:00,Red Velvet Cake,1,7.083070,South,Cash,False,Rainy,23.549497,Average,...,33,In Store,Anthony Wiggins,"24682 Holly Stravenue\nMooreville, NH 13901",769.318.4373,michellefernandez@example.com,Sarah Bentley,ajohnson@example.net,Sunday,7.083070
2,2023-01-01 02:00:00,Chocolate Cake,5,6.736064,North,Cash,True,Rainy,NaN,Excellent,...,43,Phone Order,Ashley Duke,10184 Washington Trace Apt. 679\nEast Brandist...,702.520.3286,cooperwilliam@example.com,Connie Cervantes,michele29@example.net,Sunday,33.680321
3,2023-01-01 03:00:00,Carrot Cake,2,7.314823,North,Cash,False,Cloudy,20.137483,Poor,...,32,Online Order,Brandon Taylor,"87194 Jeff Rue\nMitchellbury, CA 50463",622-527-9530,fsilva@example.net,Jessica Stewart,xwilson@example.org,Sunday,14.629647
4,2023-01-01 04:00:00,Pizza Pepperoni,1,7.577727,East,Credit Card,True,Cloudy,23.020987,Good,...,58,In Store,Brittany Watkins,"850 Julia Groves\nHartview, WI 95954",759-517-8359,petersjoseph@example.net,Cheryl Carpenter,christine96@example.org,Sunday,7.577727


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1005 entries, 0 to 1004
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Date                    1005 non-null   object 
 1   ProductName             1005 non-null   object 
 2   Quantity                1005 non-null   int64  
 3   UnitPrice               1005 non-null   float64
 4   StoreLocation           1005 non-null   object 
 5   PaymentType             1005 non-null   object 
 6   PromotionApplied        1005 non-null   bool   
 7   Weather                 1005 non-null   object 
 8   Temperature             904 non-null    float64
 9   StaffPerformanceRating  1005 non-null   object 
 10  CustomerFeedback        905 non-null    object 
 11  DeliveryTime_min        1005 non-null   int64  
 12  OrderType               1005 non-null   object 
 13  CustomerName            1005 non-null   object 
 14  CustomerAddress         1005 non-null   

In [6]:
# Data cleaning

data.drop_duplicates(inplace = True)

In [7]:
 # Handle missing values ( fillinf missing numeric values with the mean or median)
numeric_columns = data.select_dtypes(include=['float64', 'int64']).columns
for col in numeric_columns:
        data.fillna({col: data[col].mean()}, inplace=True)



In [8]:
# Handle missing values (fill missing string/object values with 'Unknown')
string_columns = data.select_dtypes(include=['object']).columns
for col in string_columns:
    data.fillna({col: 'Unknown'}, inplace=True)

In [9]:

# Cleaning date column: assigning the right data type
data['Date'] = pd.to_datetime(data['Date'])


Data Modelling

In [10]:


# Creating fact and dimension tables
# Create the product table
products = data[['ProductName']].drop_duplicates().reset_index(drop=True)
products.index.name = 'ProductID'
products = products.reset_index()

# Create Customers Table
customers = data[['CustomerName', 'CustomerAddress', 'Customer_PhoneNumber', 'CustomerEmail']].drop_duplicates().reset_index(drop=True)
customers.index.name = 'CustomerID'
customers = customers.reset_index()

# staff table['Staff_Name', 'Staff_Email']
staff = data[['Staff_Name', 'Staff_Email']].drop_duplicates().reset_index(drop=True)
staff.index.name = 'StaffID'
staff = staff.reset_index()

# Creating the transaction table
transaction = data.merge(products, on=['ProductName'], how='left') \
                .merge(customers, on=['CustomerName', 'CustomerAddress', 'Customer_PhoneNumber', 'CustomerEmail'], how='left') \
                .merge(staff, on=['Staff_Name', 'Staff_Email'], how='left') 

transaction.index.name = 'TransactionID'
transaction = transaction.reset_index() \
                        [['Date', 'TransactionID', 'ProductID',  'Quantity', 'UnitPrice', 'StoreLocation', 'PaymentType', 'PromotionApplied', 'Weather', \
                            'Temperature', 'StaffPerformanceRating', 'CustomerFeedback', 'DeliveryTime_min', 'OrderType', 'CustomerID', 'StaffID', \
                            'DayOfWeek', 'TotalSales']]

    

In [11]:
# Save data as csv files
data.to_csv('data/clean_data.csv', index=False)
products.to_csv('data/products.csv', index=False)
customers.to_csv('data/customers.csv', index=False)
staff.to_csv('data/staff.csv', index=False)
transaction.to_csv('data/transaction.csv', index=False)

print('Data Cleaning and Transformation completed successfully')

Data Cleaning and Transformation completed successfully


In [12]:
# Data Loading
def run_loading():
    # Loading the dataset
    data = pd.read_csv(r'data/cleaneddata.csv')
    products = pd.read_csv(r'data/products.csv')
    staff = pd.read_csv(r'data/staff.csv')
    customers = pd.read_csv(r'data/customers.csv')
    transaction = pd.read_csv(r'data/transaction.csv')

   

   

   

In [13]:
 # Load the environment variables from the .env files
load_dotenv()

connect_str = os.getenv('AZURE_CONNECTION_STRING_VALUE')
container_name = os.getenv('CONTAINER_NAME')

In [14]:
 # Create a BlobServiceClient object
blob_service_client = BlobServiceClient.from_connection_string(connect_str)
container_client = blob_service_client.get_container_client(container_name)

In [15]:
 # Load data to Aure Blob Storage
files = [
    (data, 'rawdata/cleaned_zipco_transaction_data.csv'),
    (products, 'cleaneddata/products.csv'),
    (customers, 'cleaneddata/customers.csv'),
    (staff, 'cleaneddata/staff.csv'),
    (transaction, 'cleaneddata/transaction.csv')
]

for file, blob_name in files:
    blob_client = container_client.get_blob_client(blob_name)
    output = file.to_csv(index=False)
    blob_client.upload_blob(output, overwrite=True)
    print(f'{blob_name} loaded into Azure Blob Storage')

rawdata/cleaned_zipco_transaction_data.csv loaded into Azure Blob Storage
cleaneddata/products.csv loaded into Azure Blob Storage
cleaneddata/customers.csv loaded into Azure Blob Storage
cleaneddata/staff.csv loaded into Azure Blob Storage
cleaneddata/transaction.csv loaded into Azure Blob Storage
